In [7]:
!pip install split-folders

In [15]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import warnings

#warnings.filterwarnings('ignore')


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])

    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/images-highpassfilter-small-zip/IMAGES_HighPassFilter_Small"
    output_folder = r"/kaggle/output/Images"

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    #splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/output/Images/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()


    # Define the VGG model
    model = torchvision.models.vgg16(pretrained=True)
    num_features = model.classifier[0].in_features
    model.classifier = nn.Sequential(
        nn.Linear(num_features, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, len(list_of_classes))
    )

    ## uncomment for CPU
    model = model.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    START_LR = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=START_LR)
    model = model.to(device)
    criterion = criterion.to(device)

    ### uncomment for GPU
    model.cuda()
        # Train the model
    for epoch in range(10):
        ### uncomment for GPU
        torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch + 1, 10))
        print('-' * 10)

        running_loss = 0
        running_corrects = 0

        model.train()
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        print("accuracy for all classes in train phase", ACC)
        print("recall for all classes in train phase", TPR)
        print("precision for all classes in train phase", PPV)
        
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-train-epoch' + str(epoch + 1) + '.csv')
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
        
        # Validation phase
        running_loss = 0
        running_corrects = 0
        model.eval()  # set the model to evaluation mode
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
       
        #classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')
        #matrix = confusion_matrix(true_labels, predictions)
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-val-epoch' + str(epoch + 1) + '.csv')
        print("accuracy for all classes in validation phase", ACC)
        print("recall for all classes in validation phase", TPR)
        print("precision for all classes in validation phase", PPV)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
                
        

    # Save the model
    torch.save(model, 'vgg16_model.pth')




['fotdro5', 'fislov1', 'hadibi1', 'pabspa1', 'gobbun1', 'whbwea1', 'gbesta1', 'yebere1', 'categr', 'afrthr1', 'grwpyt1', 'chucis1', 'grbcam1', 'mabeat1', 'scrcha1', 'bkctch1', 'vimwea1', 'mouwag1', 'wtbeat1', 'whbwhe3', 'hunsun2', 'whbtit5', 'kvbsun1', 'varsun2', 'squher1', 'blhgon1', 'afrgrp1', 'crheag1', 'blnwea1', 'stusta1']
Epoch 1/10
----------


100%|██████████| 1388/1388 [02:08<00:00, 10.79it/s]


accuracy for all classes in train phase [0.99034443 0.92765528 0.91194697 0.98472402 0.99135322 0.75616083
 0.98731806 0.97982418 0.99409137 0.67026949 0.96253062 0.92160254
 0.90286785 0.9877504  0.92318778 0.99596484 0.99567661 0.98962387
 0.97261853 0.99639718 0.97247442 0.98991209 0.99711774 0.91670269
 0.98731806 0.98962387 0.99250612 0.97809483 0.98717394 0.9711774 ]
recall for all classes in train phase [0.         0.38961039 0.1838843  0.         0.         0.65714286
 0.         0.00714286 0.         0.59786822 0.00793651 0.29245283
 0.06640625 0.         0.37575758 0.         0.         0.
 0.14457831 0.         0.04848485 0.         0.         0.43428571
 0.         0.         0.         0.11643836 0.         0.06586826]
precision for all classes in train phase [       nan 0.45       0.29180328        nan        nan 0.322
        nan 0.5               nan 0.24779116 0.16666667 0.33695652
 0.14782609 0.         0.45365854 0.                nan        nan
 0.33333333        na

100%|██████████| 350/350 [00:11<00:00, 30.48it/s]


accuracy for all classes in validation phase [0.99028571 0.97028571 0.89371429 0.98457143 0.99085714 0.91714286
 0.98742857 0.98       0.99371429 0.86       0.96342857 0.89657143
 0.87714286 0.988      0.88571429 0.996      0.99542857 0.98971429
 0.96228571 0.996      0.97771429 0.98971429 0.99714286 0.94228571
 0.98742857 0.98914286 0.992      0.98685714 0.98057143 0.976     ]
recall for all classes in validation phase [0.         0.69827586 0.67768595 0.         0.         0.47560976
 0.         0.02777778 0.         0.62015504 0.         0.70093458
 0.25       0.         0.91129032 0.         0.         0.
 0.80952381 0.         0.0952381  0.         0.         0.66666667
 0.         0.         0.         0.59459459 0.47826087 0.        ]
precision for all classes in validation phase [       nan 0.82653061 0.3580786         nan        nan 0.87969925
        nan 1.                nan 0.52117264        nan 0.33482143
 0.21192053        nan 0.37417219        nan        nan        nan
 

...

accuracy for all classes in validation phase [0.98685714 0.98171429 0.97257143 0.992      0.99314286 0.96914286
 0.98171429 0.99085714 0.99371429 0.96       0.98342857 0.98857143
 0.98514286 0.98742857 0.98057143 0.99942857 0.996      0.99771429
 0.996      0.99828571 0.99028571 0.99257143 0.99771429 0.97771429
 0.98685714 0.99257143 0.99485714 0.99542857 0.99657143 0.988     ]
recall for all classes in validation phase [0.70588235 0.93103448 0.71900826 0.62962963 0.625      0.92276423
 0.72727273 0.94444444 0.45454545 0.89922481 0.734375   0.94392523
 0.8515625  0.80952381 0.74193548 0.85714286 0.125      0.88888889
 0.88095238 0.71428571 0.76190476 0.61111111 0.8        0.81818182
 0.36363636 0.68421053 0.35714286 0.86486486 0.73913043 0.76190476]
precision for all classes in validation phase [0.4        0.81818182 0.86138614 0.80952381 0.625      0.86641221
 0.38095238 0.70833333 0.5        0.84057971 0.79661017 0.87826087
 0.93965517 0.48571429 0.9787234  1.         1.         0.88

100%|██████████| 1388/1388 [02:08<00:00, 10.79it/s]


accuracy for all classes in train phase [0.99726185 0.99437959 0.98587693 0.99812653 0.99726185 0.98702983
 0.99682951 0.99769419 0.99855887 0.98342701 0.99034443 0.99149733
 0.99394725 0.99726185 0.9900562  0.99855887 0.99783831 0.99913532
 0.99812653 0.99927944 0.99538838 0.99524427 0.99942355 0.99048854
 0.99538838 0.99755008 0.99740597 0.99841476 0.99783831 0.99481193]
recall for all classes in train phase [0.86567164 0.95238095 0.89049587 0.9245283  0.86666667 0.95816327
 0.88636364 0.95       0.87804878 0.95445736 0.86507937 0.93396226
 0.95703125 0.86746988 0.92727273 0.76923077 0.7        0.94444444
 0.96987952 0.92       0.89090909 0.68571429 0.9        0.94285714
 0.84090909 0.86111111 0.80769231 0.95890411 0.8988764  0.89820359]
precision for all classes in train phase [0.85294118 0.96280088 0.90546218 0.95145631 0.82539683 0.95040486
 0.86666667 0.93661972 0.87804878 0.9354226  0.8685259  0.92740047
 0.96078431 0.9        0.93292683 0.83333333 0.77777778 0.97142857
 0.95266

100%|██████████| 350/350 [00:11<00:00, 30.59it/s]


accuracy for all classes in validation phase [0.98571429 0.98685714 0.976      0.99314286 0.99371429 0.96228571
 0.99257143 0.98914286 0.99314286 0.96171429 0.97657143 0.98571429
 0.98171429 0.99314286 0.96742857 0.99942857 0.996      0.992
 0.99714286 0.99714286 0.99142857 0.992      0.996      0.97257143
 0.988      0.99085714 0.99657143 0.99542857 0.99428571 0.99314286]
recall for all classes in validation phase [0.64705882 0.85344828 0.72727273 0.77777778 0.75       0.95934959
 0.63636364 0.47222222 0.         0.89534884 0.78125    0.8317757
 0.78125    0.52380952 0.88709677 0.85714286 0.125      0.94444444
 0.88095238 0.85714286 0.76190476 0.38888889 1.         0.86363636
 0.40909091 0.57894737 0.71428571 0.86486486 0.7826087  0.78571429]
precision for all classes in validation phase [0.36666667 0.94285714 0.90721649 0.77777778 0.63157895 0.80821918
 0.73684211 1.         0.         0.85239852 0.64935065 0.92708333
 0.96153846 0.84615385 0.71895425 1.         1.         0.56666667